<a href="https://colab.research.google.com/github/PowerChure/Lender-Extraction-PDF/blob/main/S_G_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# libraries used
!pip install tabula-py
!pip install PyMuPDF
!pip install google.colab

from google.colab import files
import os

# importing libraries
import re
import tabula
import pandas as pd
import numpy as np
import fitz #PyMuPDF

# Upload a file

In [ ]:
def upload_file():
  """Uploads a file to the Colab environment.

  Returns:
    The path of the uploaded file.
  """
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

  return fn

#Using Airtable API rest on python to send data to Airtable

In [ ]:
import requests
import pandas as pd
import json
import os

'''# Replace with your actual Airtable API key and base ID
AIRTABLE_API_KEY = "pathzdB6rVeNqR1Lg.f64e47074424301278f578aed1151c7c30ba99ef241fef45b6da377f470b44b4" #"patcTVfyyZrTfWz6f.91c81c1e81bb6d51da7fe6398817ac39df031b7c3b20dbdce8c7fc7813996b6c"
AIRTABLE_BASE_ID = "appJpB3ee6h4i7jAM"
AIRTABLE_TABLE_NAME = "tblDyLDKs46ZjFLzU" #"tblPQ6gpAMmy0jY8G"'''

# Replace with your actual Airtable API key and base ID
AIRTABLE_API_KEY = "pathzdB6rVeNqR1Lg.fb16fa9a64cf5f9a8b3e00bd66b01d1073f08ddd7f75cf9b6741e8da4695ae6e"
AIRTABLE_BASE_ID = "appZdjg7DRvDEzYTf"
AIRTABLE_TABLE_NAME = "tbl5EOtiZxA7mLDr9"

def fetch_table_fields(api_key, base_id, table_name):
    """
    Fetches all fields (columns) for a specific table in an Airtable base using the Schema API.

    Args:
        api_key (str): Your Airtable API key.
        base_id (str): Your Airtable base ID.
        table_name (str): The name of the table in Airtable.

    Returns:
        list: A list of field names in the specified Airtable table.
    """
    url = f"https://api.airtable.com/v0/meta/bases/{base_id}/tables"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        tables = response.json().get("tables", [])
        #print(f'tables: {tables}')

        # Find the table and return its fields
        for table in tables:
            #print(f'table: {table}')
            if table["id"] == table_name:
                return [field["name"] for field in table["fields"]]

        print(f"Table '{table_name}' not found in the base.")
        return []

    except requests.exceptions.RequestException as e:
        print(f"Error fetching table fields: {e}")
        return []

def send_dataframe_to_airtable_(df, api_key, base_id, table_name):
    """
    Sends a Pandas DataFrame to an Airtable table, aligning columns with Airtable fields.

    Args:
        df: The DataFrame to send.
        api_key: Your Airtable API key.
        base_id: Your Airtable base ID.
        table_name: The name of the Airtable table.
    """
    # Fetch all fields from the Airtable table
    airtable_fields = fetch_table_fields(api_key, base_id, table_name)
    if not airtable_fields:
        print(f"Unable to retrieve fields for table '{table_name}'. Exiting.")
        return

    # Filter DataFrame to include only columns that exist in Airtable
    #print(f'df columns: {df.columns}')
    df = df[[col for col in df.columns if col in airtable_fields]]
    #print(f'df: {df.head}')

    if df.empty:
        print("No matching columns between DataFrame and Airtable table.")
        return

    url = f"https://api.airtable.com/v0/{base_id}/{table_name}"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    # Send data in batches of 10 records
    for i in range(0, len(df), 10):
        records = []
        for j in range(i, min(i + 10, len(df))):
            fields = df.iloc[j, :].dropna().to_dict()

            records.append({"fields": fields})

        payload = {"records": records}
        print(f'payload: {payload}')

        try:
            response = requests.post(url, headers=headers, data=json.dumps(payload))
            response.raise_for_status()  # Raise an exception for bad status codes

            print(f"Successfully sent data to Airtable: {response.status_code}")
            print(response.json())
        except requests.exceptions.RequestException as e:
            print(f"Error sending data to Airtable: {e}")
            print(f"Response content: {response.content}")



# Data Extraction from Scottsman pages

In [ ]:
def extract_tables_from_pdf_mow(pdf_path):
  # Initialize an empty list to store DataFrames
  dataframes = []

  # Define a maximum number of columns for standardization
  max_columns = 0

  #verify if there is the table of lenders
  get_in = False

  # regex of the date
  date_pattern = r"((January|February|March|April|May|June|July|August|September|October|November|December) \d{4})"

  # Open the PDF document
  with fitz.open(pdf_path) as doc:
    target_pages = []
    Lender_Listings = {}

    #this is for identify the date of the magazine
    first_page = doc.load_page(0)  # load the first page
    text_page = first_page.get_text("text",sort = True) #identify text and sort it
    date = re.search(date_pattern,text_page)
    j = 1
    while date is None and j < len(doc):
      first_page = doc.load_page(j)  # load the first page
      text_page = first_page.get_text("text",sort = True)
      date = re.search(date_pattern,text_page)
      j += 1

    print(f"date of scotsman magazine: {date.group(0)}")

    for page in doc:
        tablas = page.find_tables()  # Search for tables on the page
        if not tablas:  # Skip pages with no tables
            continue

        if tablas.tables:  # If tables are found
            for i, table in enumerate(tablas.tables):
                print(f'Page number: {page.number + 1}')
                print(f'Table {i + 1}:')

                # Convert the table to a DataFrame
                df = table.to_pandas()

                #assignaments
                value = set('Lender Name and Contact Info')
                test = set(df.columns[0])

                # Check if 'Lender Name and Contact Info' is in any cell of the DataFrame or if it is on the first column
                if df.astype(str).apply(lambda x: x.str.contains('Lender Name and Contact Info', case=False, na=False)).any().any() or value == test:
                  target_pages.append(page.number + 1)
                  page = doc.load_page(page.number)  # number of page
                  page_text = page.get_text("text",sort = True) #identify text and sort it
                  words = page_text.splitlines()
                  get_in = True

                  # Check if words is not empty and words[0] is not None before applying regex
                  if words and words[0] is not None:
                      palabra = re.search(r"([A-Z][a-z]+\s)?[A-Z][a-z]{4,}\s?$",str(words[0])) #extract all header words
                      i = 1
                      # Check if palabra is not None before accessing group
                      while palabra is None and i < len(words):
                          palabra = re.search(r"([A-Z][a-z]+\s)?[A-Z][a-z]{4,}\s?$",str(words[i])) #extract all header words
                          i += 1

                      Lender_Listings[page.number+1] = palabra.group(0) # add to dicctionary

                      # Insert a new column with the lender type for rows with non-empty first column
                      df.insert(2,'Type Lender','') # Initialize the column with empty strings
                      non_empty_rows = df.iloc[:,0].notna()  # Get a boolean mask of rows where the first column is not empty
                      df.loc[non_empty_rows, 'Type Lender'] = palabra.group(0)  # Assign the lender type to these rows

                  else:
                      print(f"Warning: No words found on page {page.number + 1}. Skipping this page.")


                # Skip tables with fewer than 6 columns
                if get_in is False or len(df.columns) < 6:
                    print(f"Table {i + 1} on page {page.number + 1} is not a lender list")
                    continue

                # Reset the columns to numerical indices (0, 1, 2, ...)
                df.columns = range(len(df.columns))

                # Update the max_columns based on the current table
                max_columns = max(max_columns, len(df.columns))

                # Add the DataFrame to the list
                dataframes.append(df)

  # add the date to lender listings
  Lender_Listings[date[0]] = "date of magazine"
  # show the lender listings
  print(f"number of page:type lender => {Lender_Listings}")
  # Normalize the number of columns across all DataFrames
  normalized_dataframes = []
  for i,df in enumerate(dataframes):
    # Ensure the DataFrame has `max_columns` by filling with empty strings
    missing_cols = max_columns - len(df.columns)
    if missing_cols > 0:
        # Add missing columns
        for _ in range(missing_cols):
            df[len(df.columns)] = ""  # Add empty columns

    # Ensure columns are properly ordered by resetting their names
    df.columns = range(max_columns)
    #df.insert(max_columns,max_columns,type_lender[i])
    normalized_dataframes.append(df)

  # Combine all normalized DataFrames into one
  if normalized_dataframes:
    final_df = pd.concat(normalized_dataframes, ignore_index=True)

    # Save to a CSV file
    final_df.to_csv("extracted_tables.csv", index=False)
    print("Tables have been saved to 'extracted_tables.csv'")
  else:
    print("No tables with 6 or more rows found in the PDF.")

  return final_df,Lender_Listings

In [ ]:
def Process_sparse_rows(df):
# Process sparse rows to merge single-value rows into the previous row
  df2 = df
  for row_idx in range(1, len(df2)):
    row = df2.iloc[row_idx]
    non_empty_values = row[row != ""].dropna().to_list()  # Get non-empty values
    # Check if the row contains only one non-empty value
    if len(non_empty_values) == 1:
        sparse_value = non_empty_values[0]  # Extract the single non-empty value

        # Display the sparse row and the value detected
        print(f"Row {row_idx} is sparse. Detected value: '{sparse_value}'")

        # Find the first non-empty column in the previous row and append the sparse value
        for col_idx in range(len(df2.columns)):
            if df2.iloc[row_idx - 1, col_idx] != "":
                df2.iloc[row_idx - 1, col_idx] = (
                    str(df2.iloc[row_idx - 1, col_idx]) + '\n' + sparse_value
                )
                print(f"Appended sparse value to Row {row_idx - 1}, Column {col_idx}")
                break

        # Clear the current sparse row after moving its value
        df2.iloc[row_idx] = [""] * len(row)
        print(f"Row {row_idx} cleared after processing.")

  print(df2)
  # Save to a CSV file
  df2.to_csv("extracted_tables_sparsed.csv", index=False)
  print("Tables have been saved to 'extracted_tables_sparsed.csv'")


# Data Processing - classify each datum within Lender Name, Website, Contact Number, Email, and Type Lender, afterward transfer to a CSV (deal with missing data)

In [ ]:
def rearrange(csv_file,dicto):
  ''' this program rearrange data for upload to airtable'''

  #here the regex patterns
  website_pattern = r"(?:www\.)?[\w-]+\.[\w.-]+"  # Matches websites
  contact_number_pattern = r"^([A-Z][a-z]+(\s+[A-Z].+)?)?\s*(\w+@\w+\.\w+)?\s*\((\d{3}\)|\d{3})\s?\d{3}-\d{4}|^[A-Z][a-z]+(\s+[A-Z]\w{,23})?$" # Matches contact numbers like "Jhon Dan (###) ###-####" and includes email
  email_pattern = r"[\w\.-]+@[\w\.-]+\.\w{2,5}"  # Matches email addresses
  min_max_pattern = r'(^\d+[KMB]|^0\s.*)'
  lending_territory_pattern = r'^Lending Territory: .+'

  #obtain the dataframe cleaning
  df = pd.read_csv(csv_file)
  df = df.dropna(how='all').reset_index(drop=True)

  num = int(df.columns.get_loc(df.columns[df.isin(['Min/Max']).any()].tolist()[0])) #get the number of column where is Min\Max amounts

  indexes = [] # will collect all those lender name and info rows

  names = []
  websites = []
  contact_numbers = []
  emails = []
  lender_terr = []
  except_terr = []
  infos = []

  #to assign the type of lender
  type_lender = list(dicto.keys())
  print(f'type of lender size: {len(type_lender)}, list {type_lender}')

  for index, row in df.iterrows():
    first_contain = str(df.iloc[index,0]) # here is the first cell of each row
    first_contain = first_contain.split('\n')
    print(f'size {len(first_contain)}: {first_contain}')

    if 1 < len(first_contain) < 7 :
      # Initialize data fields
      verifyer = {
        'lender_name' : ""
        ,'website' : ""
        ,'contact_number' : ""
        ,'email' : ""
        ,'Lender Territory' : ""
        ,'Exception Territory' : ""
        ,'Info' : ""
        }
      i = 0
      while any([verifyer['lender_name'] == "",verifyer['website'] == "",verifyer['contact_number'] == "",verifyer['email'] == "",verifyer['Lender Territory'] == "",verifyer['Exception Territory'] == "",verifyer['Info'] == ""]):
              try:
                if i < len(first_contain) and isinstance(first_contain[i], str):

                  if i == 0 and not re.match(r'[A-Z]{6,}( ([A-Z]{2,}))?',first_contain[i]):
                        verifyer['lender_name'] = first_contain[i]
                        i += 1

                  elif re.match(website_pattern,first_contain[i]):
                        verifyer['website'] = first_contain[i]
                        i += 1

                  elif re.match(contact_number_pattern,first_contain[i]):
                        verifyer['contact_number'] = first_contain[i]
                        i += 1

                  elif re.match(email_pattern,first_contain[i]):
                        verifyer['email'] = first_contain[i]
                        i += 1

                  elif re.match(lending_territory_pattern,first_contain[i]): # cheking if there is Except territory
                        new = re.sub(r'^Lending Territory: ',"",first_contain[i])
                        if 'Except:' in new:
                          territory = new.split('Except: ')
                          verifyer['Exception Territory'] = territory[1]
                          verifyer['Lender Territory'] = territory[0]
                        elif 'except' in new:
                          territory = new.split('except')
                          verifyer['Exception Territory'] = territory[1]
                          verifyer['Lender Territory'] = territory[0]
                        else:
                          verifyer['Lender Territory'] = new
                        i += 1
                  else:
                        verifyer['Info'] = first_contain[i]
                        i += 1

                elif i < len(first_contain):
                    print(f"Unexpected data type: {type(first_contain[i])}")
                else:
                    break
              except Exception as e:
                print(f"Error processing item: {first_contain[i]}, Error: {e}")

              #detect where are missing data and save the lender rows
              if i == len(first_contain) and any([verifyer['lender_name'] == "",verifyer['website'] == "",verifyer['contact_number'] == "",verifyer['email'] == "",verifyer['Lender Territory'] == "",verifyer['Exception Territory'] == "",verifyer['Info'] == ""]):
                for key in ['lender_name','website','contact_number','email','Lender Territory','Exception Territory','Info']: #replace each empty place with "missing"
                  if verifyer[key] == "":
                    verifyer[key] = "missing"
              if i == len(first_contain) and verifyer['lender_name'] != "missing":
                  indexes.append(index)
                  # Append lender details ONLY when a lender row is identified
                  names.append(verifyer['lender_name'])
                  websites.append(verifyer['website'])
                  contact_numbers.append(verifyer['contact_number'])
                  emails.append(verifyer['email'])
                  lender_terr.append(verifyer['Lender Territory'])
                  except_terr.append(verifyer['Exception Territory'])
                  infos.append(verifyer['Info'])

              print(f'here on the while {verifyer}')
              print(f'iters: {i}')

  print(f'size {len(indexes)} numbers of rows {indexes}')
  df = df.drop(df.columns[0], axis=1)

  #it turns the data from float to string
  #df.iloc[:,num+1:] = df.iloc[:,num+1:].astype(str)

  #change the name of columns
  if df.columns[num-1] == df.columns[2]:
    # Create a new list of column names
    new_columns = ['Type','Type Lender','Min/Max'] + [str(i) for i in range(1, len(df.columns) - 2)]
    df.columns = new_columns  # Assign the new list to df.columns
  else:
    # Create a new list of column names
    new_columns = ['Min/Max','Type Lender'] + [str(i) for i in range(1, len(df.columns) - 1)]
    df.columns = new_columns  # Assign the new list to df.columns

  #data frame filtered
  df_shortened = df[df.index.isin(indexes)]
  #print(df_shortened.head())

  print(f'type lender {dicto}')

  # here add new columns to df
  df_shortened.insert(0, 'Lender Name',names)
  df_shortened.insert(1, 'Website',websites)
  df_shortened.insert(2, 'Contact Number',contact_numbers)
  df_shortened.insert(3, 'Email',emails)
  df_shortened.insert(6, 'Month/Year',type_lender[-1])
  df_shortened.insert(7, 'Lender Territory',lender_terr)
  df_shortened.insert(8, 'Exception Territory',except_terr)
  df_shortened.insert(9, 'Info',infos)

  # Function to convert shorthand to numbers
  def convert_to_number(value):
    multiplier = {'K': 1_000, 'M': 1_000_000, 'B': 1_000_000_000}  # Add more if needed
    if isinstance(value, str):
        if value[-1] in multiplier:
            return int(float(value[:-1]) * multiplier[value[-1]])
        else:
            return int(value)  # No shorthand, return as-is
    return None  # Handle None or missing values

  # Split 'min/max' into 'min' and 'max' columns, handling cases with no space
  split_result = df_shortened['Min/Max'].str.split('\n', expand=True, n=1)
  # Reindex to ensure 2 columns (0 and 1)
  split_result = split_result.reindex(columns=[0, 1])

  # Insert 'Min' and 'Max' columns separately
  df_shortened.insert(7, 'Min', split_result[0])  # Insert 'Min' column at position 6
  df_shortened.insert(8, 'Max', split_result[1])  # Insert 'Max' column at position 7 (after 'Min')

  # Fill NaN values in 'Max' with corresponding values from 'Min' if 'Max' is missing
  df_shortened['Max'] = df_shortened['Max'].fillna(df_shortened['Min'])

  # Ensure single values are always assigned to the 'max' column
  df_shortened['Max'] = df_shortened.apply(lambda row: row['Min'] if pd.isna(row['Max']) else row['Max'], axis=1)
  df_shortened['Min'] = df_shortened.apply(lambda row: None if row['Max'] == row['Min'] else row['Min'], axis=1)

  # Convert the shorthand values to numbers
  df_shortened['Min'] = df_shortened['Min'].apply(convert_to_number)
  df_shortened['Max'] = df_shortened['Max'].apply(convert_to_number)

  # Drop the original 'min/max' column if no longer needed
  df_shortened = df_shortened.drop(columns=['Min/Max'])

  # Export the DataFrame to a CSV file
  df_shortened.to_csv("classified_lenders.csv", index=False, encoding='utf-8')  # Specify encoding to handle special characters
  print("Data classified and exported to 'classified_lenders.csv'")

  return df_shortened

# Main Function

In [ ]:
if __name__ == "__main__":
  # Example usage
  uploaded_file_path = upload_file()
  print(f"Uploaded file path: {uploaded_file_path}")


  df,date = extract_tables_from_pdf_mow(pdf_path=uploaded_file_path)

  Process_sparse_rows(df)

  df = rearrange('extracted_tables_sparsed.csv',date)

  csv_file_path = "classified_lenders.csv"  # Replace with the actual path to your CSV file
  if os.path.exists(csv_file_path):
    df = pd.read_csv(csv_file_path)
    send_dataframe_to_airtable_(df, AIRTABLE_API_KEY, AIRTABLE_BASE_ID, AIRTABLE_TABLE_NAME)
  else:
    print(f"Error: CSV file '{csv_file_path}' not found.")

Se truncaron las últimas líneas 5000 del resultado de transmisión.
Table 1 on page 110 is not a lender list
Page number: 110
Table 2:
Table 2 on page 110 is not a lender list
Page number: 110
Table 3:
Table 3 on page 110 is not a lender list
Page number: 111
Table 1:
Table 1 on page 111 is not a lender list
Page number: 111
Table 2:
Table 2 on page 111 is not a lender list
Page number: 113
Table 1:
Page number: 114
Table 1:
Table 1 on page 114 is not a lender list
number of page:type lender => {39: 'Prime Commercial', 41: 'Prime Commercial', 43: 'Prime Commercial', 44: 'Prime Commercial', 45: 'Prime Commercial', 46: 'Prime Commercial', 47: 'Prime Commercial', 48: 'Prime Commercial', 49: 'Prime Commercial', 53: 'Multifamily Properties', 55: 'Multifamily Properties', 57: 'Multifamily Properties', 58: 'Multifamily Properties', 59: 'Multifamily Properties', 63: 'Hard Money', 65: 'Hard Money', 67: 'Hard Money', 69: 'Hard Money', 71: 'Hard Money', 73: 'Hard Money', 75: 'Hard Money', 77: 'Har

<ipython-input-2-95b9bef56a12>:330: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['90.0' '60.0' 'nan' 'nan' 'nan' '75.0' '5.0' '90.0' '65.0' '80.0' '75.0'
 'nan' 'nan' '65.0' 'nan' 'nan' 'nan' 'nan' 'nan' '5.0' '150.0' '75.0'
 'nan' '80.0' '90.0' '90.0' '90.0' 'nan' 'nan' 'nan' 'nan' '75.0' '5.0'
 'nan' 'nan' 'nan' 'nan' '0.0' 'nan' 'nan' '90.0' '90.0' 'nan' 'nan'
 '70.0' '5.0' '85.0' '65.0' 'nan' '60.0' 'nan' 'nan' 'nan' '65.0' '75.0'
 'nan' '100.0' 'nan' '5.0' '85.0' '70.0' 'nan' '60.0' 'nan' '75.0' 'nan'
 'nan' '75.0' 'nan' 'nan' 'nan' '5.0' 'nan' 'nan' 'nan' '85.0' 'nan'
 '90.0' '80.0' 'nan' 'nan' 'nan' '90.0' '65.0' '5.0' '75.0' 'nan' '75.0'
 '80.0' 'nan' '75.0' 'nan' 'nan' 'nan' 'nan' '100.0' '75.0' '5.0' '90.0'
 '90.0' '80.0' '80.0' '80.0' 'nan' 'nan' 'nan' '85.0' '85.0' '80.0' 'nan'
 'nan' '75.0' '75.0' 'nan' '5.0' 'nan' '60.0' 'nan' '125.0' 'nan' '65.0'
 '75.0' 'nan' 'nan' 'nan' 'nan' '75.0' '5.0' 'nan' '

payload: {'records': [{'fields': {'Lender Name': '5 Arch Funding Corp.', 'Website': 'www.5archfunding.com/scotsman', 'Contact Number': '(844) 497-5482', 'Email': 'info@5arch.com', 'Type': 'Direct\nLender', 'Type Lender': 'Prime Commercial', 'Month/Year': 'January 2020', 'Min': 750000.0, 'Max': 20000000.0, 'Lender Territory': 'NATIONWIDE ', 'Exception Territory': ': AK DC DE IA ID KY LA ME MI MN MT ND NE NH NM RI SD VT WI WV WY', 'Info': '5 Arch is a private mortgage company for experienced residential investors, mortgage brokers, and private lenders that need working capital.', '2': 90.0, '3': 90.0, '5': 90.0, '6': 90.0, '11': 'Y', '13': 'Y', '14': 'Y', '23': 'Y', '25': 'Y', '27': 'Y', '29': 'Y'}}, {'fields': {'Lender Name': 'A.S.K. Investments Inc.', 'Website': 'www.askinvestments.com', 'Contact Number': 'Tom Johnson (800) 834-6544', 'Email': 'askinvestmentsinc@yahoo.com', 'Type': 'Private\nLender', 'Type Lender': 'Prime Commercial', 'Month/Year': 'January 2020', 'Min': 0.0, 'Max': 50